# Union Needed NBA

Simple flow:
1. Load users and employees tables
2. Count employees per client
3. If count > 2, generate an NBA event (union-related advisory)
4. Optionally post events to backend

In [8]:
from nba_calculator.utils import CalculatorUtils, EnvConfig

In [9]:
# Configuration
ENDPOINT = EnvConfig.str("ENDPOINT", "http://127.0.0.1:8000/api/v1/internal/events/nba-calculation")
NBA_LIST_ENDPOINT = EnvConfig.str("NBA_LIST_ENDPOINT", "http://127.0.0.1:8000/api/v1/nba")
EMPLOYEE_THRESHOLD = EnvConfig.int("EMPLOYEE_THRESHOLD", 2)
NBA_DEFINITION_ID = EnvConfig.str("NBA_DEFINITION_ID", "def_union_needed")
SOURCE = EnvConfig.str("SOURCE", "calculator.union-needed.v1")
REQUEST_ID_PREFIX = EnvConfig.str("REQUEST_ID_PREFIX", "calc-union-needed")
DRY_RUN = EnvConfig.bool("DRY_RUN", False)  # Set false to POST events

In [10]:
def create_event_context(client, employee_count):
    context = {
        "client_id": client.get('client_id'),
        "employee_count": employee_count,
        "rule": f"employee_count > {EMPLOYEE_THRESHOLD}",
        "offer": "union_advisory",
        "message": "Your workforce size indicates union-related obligations may apply.",
        }
   
    return CalculatorUtils.set_context_hash(context)

def create_delete_event_context(client):
    context = {
        "client_id": client.get('client_id'),
        "rule": f"employee_count > {EMPLOYEE_THRESHOLD}",
        "offer": "union_advisory",
        "message": f"This NBA is no longer applicable to you, since employee_count < {EMPLOYEE_THRESHOLD}.",
        }
    context_hash = CalculatorUtils.calculate_hash(context)
    context["hash"] = context_hash

    return CalculatorUtils.set_context_hash(context)


In [11]:
# Count the number of employees per client
query = """
  SELECT
    c.id as client_id,c.enterprise_number, c.account_id,
    COUNT(ce.client_id) AS employee_count
  FROM clients c
  LEFT JOIN client_employees ce ON c.id = ce.client_id
  GROUP BY c.id;
"""

clients_with_count = CalculatorUtils.execute_sql_query(query)

In [12]:
new_events = []
for client in clients_with_count:
    employee_count = client.get('employee_count',0)
    client_id = client.get('id')

    if employee_count >= EMPLOYEE_THRESHOLD:
        
        context = create_event_context(client, employee_count)
        event = CalculatorUtils.create_event(client, context,nba_definition_id=NBA_DEFINITION_ID, source=SOURCE)
        
    new_events.append(event)

In [13]:
import json
print("Events to be posted:")
print(json.dumps(new_events, indent=2))

Events to be posted:
[
  {
    "event_id": "629c2c5e-0640-4404-8889-4b3530fedd54",
    "occurred_at": "2026-02-13T13:33:30.485677Z",
    "source": "calculator.union-needed.v1",
    "nba_definition_id": "def_union_needed",
    "enterprise_number": "0123456789",
    "context": {
      "client_id": "client_001",
      "employee_count": 2,
      "rule": "employee_count > 2",
      "offer": "union_advisory",
      "message": "Your workforce size indicates union-related obligations may apply.",
      "hash": "9341b6a6581c3673c1a0f255be1549ae7fc7d41daeea8b327280976991c6b707"
    },
    "account_id": "acc_123"
  },
  {
    "event_id": "4eb9b008-f2cd-4527-b415-ac6f9409ca69",
    "occurred_at": "2026-02-13T13:33:30.485888Z",
    "source": "calculator.union-needed.v1",
    "nba_definition_id": "def_union_needed",
    "enterprise_number": "9876543210",
    "context": {
      "client_id": "client_003",
      "employee_count": 2,
      "rule": "employee_count > 2",
      "offer": "union_advisory",
 

In [14]:
# Post creation events
CalculatorUtils.post_events(new_events, dry_run=DRY_RUN, endpoint=ENDPOINT, request_id_prefix=REQUEST_ID_PREFIX)

{'request_id': 'calc-union-needed-1', 'event_id': '629c2c5e-0640-4404-8889-4b3530fedd54', 'status': 202, 'response': '{"status":"accepted","event_id":"629c2c5e-0640-4404-8889-4b3530fedd54"}'}
{'request_id': 'calc-union-needed-2', 'event_id': '4eb9b008-f2cd-4527-b415-ac6f9409ca69', 'status': 202, 'response': '{"status":"accepted","event_id":"4eb9b008-f2cd-4527-b415-ac6f9409ca69"}'}


# In the UI you should now see one new event for client with enterprise_number 123456789
http://localhost:8000/ui/active-nbas?client=0123456789